In [35]:
from collections import Counter
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

def get_closest_match_vec(text, X, vec, TOPN=5):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:TOPN]
    
    return [id2word[top] for top in topn]

In [36]:
import textdistance

def get_closest_match_with_metric(text, lookup, metric=textdistance.levenshtein):
    similarities = Counter()
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(1)[0]

In [37]:
def get_closest_hybrid_match(text, X, vec, metric=textdistance.levenshtein):
    vec_matches = get_closest_match_vec(text, X, vec)
    final_metric = 0
    closest = ''
    for el in vec_matches:
        temp_metric = metric.normalized_similarity(text, el)
        if temp_metric > final_metric:
            final_metric = temp_metric
            closest = el
    return closest

In [38]:
%%time
get_closest_hybrid_match('апофиоз', X, vec)

Wall time: 96 ms


'апофеоз'

In [41]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [45]:
import re
from string import punctuation
punct = set(punctuation)

mistakes = []
total = 0

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        corr = get_closest_hybrid_match(pair[1], X, vec)
        if pair[0] != corr:
            newmistake = (pair[0], corr)
            mistakes.append(newmistake)
        total += 1

In [49]:
print('Доля ошибок:', len(mistakes)/total)

Доля ошибок: 0.17059528565721135


In [50]:
print(mistakes[:15])

[('симпатичнейшее', 'пластичнейшими'), ('шпионское', 'шпионские'), ('гламурный', 'лагерный'), ('бонда', 'банд'), ('superheadz', 'sueddeutsche'), ('clap', 'place'), ('camera', 'america'), ('получатся', 'ополчатся'), ('язычки', 'язычка'), ('очень', 'чечни'), ('милые', 'милы'), ('насчет', 'защищено'), ('чавеса', 'чавес'), ('попавшим', 'пропавшим'), ('аварийно-спасательных', 'аварийно-восстановительных')]


Видим, что частым типом ошибок является замена слова с опечаткой на слово того же корня, но в неверной грамматической форме (шпионское - шпионскИе) либо на слово другого, но близкого по буквенному составу, корня (получатся-ополчатся)